# Sequential single-point Bayesian optimisation

In this example, NUBO is used for sequential single-point optimisation. The `Hartmann6D` synthetic test function acts as a surrogate for a black box objective funtion, such as an experiment or a simulation. We use the analytical acquisiton function `UpperConfidenceBound` with $\beta = 1.96^2$ corresponding to the 95% confidence interval of the Gaussian distribution. We optimise this acquisition function with the `lbfgsb()` algorithm with 5 starts to avoid getting stuck in a local maximum. The optimisation loop is run for 40 iterations and finds a solution close the true optimum of -3.3224.

In [1]:
import torch
from nubo.acquisition import ExpectedImprovement, UpperConfidenceBound
from nubo.models import GaussianProcess, fit_gp
from nubo.optimisation import lbfgsb
from nubo.test_functions import Hartmann6D
from nubo.utils import gen_inputs
from gpytorch.likelihoods import GaussianLikelihood


# test function
func = Hartmann6D(minimise=False)
dims = func.dims
bounds = func.bounds

# training data
x_train = gen_inputs(num_points=dims*5,
                     num_dims=dims,
                     bounds=bounds)
y_train = func(x_train)

# Bayesian optimisation loop
iters = 40

for iter in range(iters):
    
    # specify Gaussian process
    likelihood = GaussianLikelihood()
    gp = GaussianProcess(x_train, y_train, likelihood=likelihood)
    
    # fit Gaussian process
    fit_gp(x_train, y_train, gp=gp, likelihood=likelihood, lr=0.1, steps=200)

    # specify acquisition function
    # acq = ExpectedImprovement(gp=gp, y_best=torch.max(y_train))
    acq = UpperConfidenceBound(gp=gp, beta=1.96**2)

    # optimise acquisition function
    x_new, _ = lbfgsb(func=acq, bounds=bounds, num_starts=5)

    # evaluate new point
    y_new = func(x_new)
    
    # add to data
    x_train = torch.vstack((x_train, x_new))
    y_train = torch.hstack((y_train, y_new))

    # print new best
    if y_new > torch.max(y_train[:-1]):
        print(f"New best at evaluation {len(y_train)}: \t Inputs: {x_new.numpy().reshape(dims).round(4)}, \t Outputs: {-y_new.numpy().round(4)}")

# results
best_iter = int(torch.argmax(y_train))
print(f"Evaluation: {best_iter+1} \t Solution: {-float(y_train[best_iter]):.4f}")


New best at evaluation 33: 	 Inputs: [0.1247 0.     0.634  0.2347 0.257  0.4351], 	 Outputs: [-1.7711]
New best at evaluation 37: 	 Inputs: [0.1196 0.0298 0.2002 0.2853 0.3047 0.5265], 	 Outputs: [-2.3202]
New best at evaluation 38: 	 Inputs: [0.1407 0.     0.2125 0.2922 0.3278 0.6806], 	 Outputs: [-2.6529]
New best at evaluation 40: 	 Inputs: [0.1356 0.     0.3436 0.3205 0.3754 0.6734], 	 Outputs: [-2.6605]
New best at evaluation 44: 	 Inputs: [0.1756 0.     0.351  0.3389 0.3135 0.637 ], 	 Outputs: [-2.8567]
New best at evaluation 45: 	 Inputs: [0.1758 0.     0.4007 0.2858 0.2995 0.6475], 	 Outputs: [-3.0165]
New best at evaluation 46: 	 Inputs: [0.1803 0.1282 0.4165 0.2719 0.299  0.6528], 	 Outputs: [-3.2702]
New best at evaluation 50: 	 Inputs: [0.205  0.1713 0.4946 0.2829 0.3034 0.6497], 	 Outputs: [-3.3072]
New best at evaluation 54: 	 Inputs: [0.2177 0.1405 0.474  0.2793 0.2996 0.6665], 	 Outputs: [-3.3091]
New best at evaluation 55: 	 Inputs: [0.2128 0.1415 0.4793 0.2709 0.3187 